Poniższy notebook ma za zadanie nauczyć sieć BiGRU o ustalonych w notebooku BiGRU_hyper_parametr_opt hiperparametrach na zbiorze złożonym z około 67 tys sekwencji aminokwasowych (ok. 14 mln aminokwasów), a następnie sprawdzić jej skuteczność (procentowa liczba dobrych predykcji) oraz precision/recall dla poszczególnych struktur.


In [1]:
import os
os.environ['KERAS_BACKEND'] = 'theano'
import keras
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, RepeatVector
from keras.optimizers import RMSprop, SGD
from sklearn.cross_validation import train_test_split
import numpy as np

Using Theano backend.
/home/kacper/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
mem_depth = 20
proteins = ['*']*int(mem_depth/2)
structures = ['*']*int(mem_depth/2)


path = "output102361.out"
with open(path) as f:
    for line in f:
        splited = line.strip().split(' ')
        if len(list(splited[1])) == len(list(splited[2])):
            proteins.extend(list(splited[1].upper()))
            proteins.extend(['*']*int(mem_depth/2))
            structures.extend(list(splited[2].upper()))
            structures.extend(['*']*int(mem_depth/2))
        
            
            
alphabet_structures = ['C', 'H', 'E', 'T', "*"]
alphabet_proteins = ['A','R','N','D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', '*']

known_proteins = []
known_structures = []
for i in range(len(proteins)):
    if proteins[i] in alphabet_proteins and structures[i] in alphabet_structures:
        known_proteins.append(proteins[i])
        known_structures.append(structures[i])
proteins = known_proteins
structures = known_structures

structures_indices = dict((c, i) for i, c in enumerate(alphabet_structures))
indices_structures = dict((i, c) for i, c in enumerate(alphabet_structures))

proteins_indices = dict((c, i) for i, c in enumerate(alphabet_proteins))
indices_proteins = dict((i, c) for i, c in enumerate(alphabet_proteins))


#Get time series
protein_blocks = []
structure = []
for i in range(0, len(proteins) - mem_depth + 1):
    protein_blocks.append(proteins[i: i + mem_depth])
    structure.append(structures[i + int(mem_depth/2) - 1])
    

#Vectorisation
X = np.zeros((len(protein_blocks), mem_depth, len(alphabet_proteins)), dtype=np.bool)
y = np.zeros((len(structure), len(alphabet_structures)), dtype=np.bool)
for i, block in enumerate(protein_blocks):
    for t, protein in enumerate(block):
        X[i, t, proteins_indices[protein]] = 1
    y[i, structures_indices[structure[i]]] = 1
    
 

In [3]:
print('Build model...')
model = Sequential()
model.add(Bidirectional(GRU(64), input_shape=(mem_depth, len(alphabet_proteins))))
model.add(Dense(len(alphabet_structures)))
model.add(Activation('softmax'))

optimizer = SGD(lr=0.1, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


Build model...


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, train_size=0.66)

train_filtered_x = np.array([X_train[i] for i in range(len(X_train)) if alphabet_structures[np.argmax(y_train[i])] != '*'])
train_filtered_y = np.array([y_train[i] for i in range(len(y_train)) if alphabet_structures[np.argmax(y_train[i])] != '*'])

test_filtered_x = np.array([X_test[i] for i in range(len(X_test)) if alphabet_structures[np.argmax(y_test[i])] != '*'])
test_filtered_y = np.array([y_test[i] for i in range(len(y_test)) if alphabet_structures[np.argmax(y_test[i])] != '*'])

model.fit(train_filtered_x, train_filtered_y, batch_size=128, epochs=5)

Epoch 1/5
13974594/13974594 [==============================] - 4854s - loss: 0.9025  
Epoch 2/5
13974594/13974594 [==============================] - 4939s - loss: 0.8547  
Epoch 3/5
13974594/13974594 [==============================] - 5065s - loss: 0.8454  
Epoch 4/5
13974594/13974594 [==============================] - 5030s - loss: 0.8414  
Epoch 5/5
13974594/13974594 [==============================] - 5042s - loss: 0.8391  


In [11]:
model.fit(train_filtered_x, train_filtered_y, batch_size=128, epochs=5)

Epoch 1/5
13974594/13974594 [==============================] - 44158s - loss: 0.8375 
Epoch 2/5
13974594/13974594 [==============================] - 5101s - loss: 0.8362  
Epoch 3/5
13974594/13974594 [==============================] - 5554s - loss: 0.8353  
Epoch 4/5
13974594/13974594 [==============================] - 5424s - loss: 0.8344  
Epoch 5/5
13974594/13974594 [==============================] - 5230s - loss: 0.8351  


In [18]:
#Saving trained model
model_yaml = model.to_yaml()
with open("BiGRU.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
    
model.save_weights("BiGRU.h5")
print("Saved model to disk")

Saved model to disk


In [12]:
#Predicting
predictions = (model.predict(test_filtered_x))
#Getting structure codes
predictions = np.array([alphabet_structures[np.argmax(prediction)] for prediction in predictions])
#Getting structure codes
structures = np.array([alphabet_structures[np.argmax(y)] for y in test_filtered_y])

In [13]:
#Getting accuracy
acc_arr = predictions == structures
np.mean(acc_arr)

0.66463244956326817

In [14]:
C_arr_pred = predictions == "C"
C_arr_struct = structures == "C"
C_precision = sum(C_arr_pred * acc_arr)/sum(C_arr_pred)
C_recall = sum(C_arr_pred * acc_arr)/sum(C_arr_struct)
print("C Precision: {0}".format(C_precision))
print("C Recall: {0}".format(C_recall))

C Precision: 0.6231842161956342
C Recall: 0.6054776716609714


In [15]:
H_arr_pred = predictions == "H"
H_arr_struct = structures == "H"
H_precision = sum(H_arr_pred * acc_arr)/sum(H_arr_pred)
H_recall = sum(H_arr_pred * acc_arr)/sum(H_arr_struct)
print("H Precision: {0}".format(H_precision))
print("H Recall: {0}".format(H_recall))

H Precision: 0.7096588454494616
H Recall: 0.842855394698064


In [16]:
E_arr_pred = predictions == "E"
E_arr_struct = structures == "E"
E_precision = sum(E_arr_pred * acc_arr)/sum(E_arr_pred)
E_recall = sum(E_arr_pred * acc_arr)/sum(E_arr_struct)
print("E Precision: {0}".format(E_precision))
print("E Recall: {0}".format(E_recall))

E Precision: 0.6624632165823134
E Recall: 0.6477236271411647


In [17]:
T_arr_pred = predictions == "T"
T_arr_struct = structures == "T"
T_precision = sum(T_arr_pred * acc_arr)/sum(T_arr_pred)
T_recall = sum(T_arr_pred * acc_arr)/sum(T_arr_struct)
print("T Precision: {0}".format(T_precision))
print("T Recall: {0}".format(T_recall))

T Precision: 0.5469333950701863
T Recall: 0.2869914355521893
